<a href="https://colab.research.google.com/github/choudaryhussainali/MCQ_Grading_Bot/blob/main/MCQ_Grading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio Pillow groq google-generativeai python-dotenv --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.0 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

# For Groq
GROQ_API_KEY = userdata.get('GROQ_API_KEY')  # Set this in Colab secrets

# For Gemini
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')  # Set this in Colab secrets


In [3]:
import gradio as gr
from PIL import Image
import io
import time
import json
from google.colab import userdata
import google.generativeai as genai

# Configure Gemini
genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
model = genai.GenerativeModel('gemini-1.5-flash')

def grade_mcqs_from_image(image):
    start_time = time.time()

    # Prepare the image
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_bytes = img_byte_arr.getvalue()

    # Optimized prompt for Gemini Vision
    prompt = """Carefully analyze this image of a solved MCQ exam paper. Extract and return the following information in JSON format:

    {
        "student_name": "[Name from paper or 'Unknown']",
        "total_questions": [Total number of questions],
        "correct_answers": [Number of correct answers],
        "wrong_answers": [Number of wrong answers],
        "score_percentage": [Percentage score],
        "mcqs": [
            {
                "question_number": "[Q1, Q2, etc.]",
                "question": "[Full question text]",
                "correct_answer": "[Correct option]",
                "student_answer": "[Student's chosen option]",
                "is_correct": [true/false]
            },
            ... (repeat for all questions)
        ]
    }

    Important guidelines:
    1. Focus on accuracy - double-check each answer
    2. For student_name, look for any name written on the paper
    3. Clearly distinguish between correct answers and student answers
    4. Return ONLY valid JSON, no additional text or explanations
    """

    try:
        # Send to Gemini for processing
        response = model.generate_content(
            [prompt, Image.open(io.BytesIO(img_bytes))],
            generation_config={
                "temperature": 0.1,
                "max_output_tokens": 4000
            }
        )

        # Clean the response to extract pure JSON
        response_text = response.text.replace('```json', '').replace('```', '').strip()
        result = json.loads(response_text)

        # Calculate processing time
        processing_time = time.time() - start_time

        # Format the results for display
        formatted_output = f"⏱ Processed in {processing_time:.2f} seconds\n\n"
        formatted_output += f"👤 Student: {result.get('student_name', 'Unknown')}\n"
        formatted_output += f"📝 Total Questions: {result['total_questions']}\n"
        formatted_output += f"✅ Correct: {result['correct_answers']} | ❌ Wrong: {result['wrong_answers']}\n"
        formatted_output += f"📊 Score: {result['score_percentage']}%\n\n"
        formatted_output += "Detailed Results:\n"

        for mcq in result['mcqs']:
            status = "✓" if mcq['is_correct'] else "✗"
            formatted_output += f"\n{mcq['question_number']}. {mcq['question']}\n"
            formatted_output += f"   Correct: {mcq['correct_answer']} | Student: {mcq['student_answer']} {status}\n"

        return formatted_output

    except Exception as e:
        return f"❌ Error processing image: {str(e)}\n\nPlease ensure:\n1. Image is clear and well-lit\n2. Answers are clearly marked\n3. Text is readable"

# Create the Gradio interface
with gr.Blocks(title="MCQ Grading Bot") as demo:
    gr.Markdown("## 📝 Automated MCQ Grading with Gemini Vision")
    gr.Markdown("Upload an image of a solved MCQ answer sheet for automatic grading")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(label="Upload Answer Sheet", type="pil")
            submit_btn = gr.Button("Grade Answers", variant="primary")

        with gr.Column():
            output = gr.Textbox(label="Grading Results", lines=20, interactive=False)

    submit_btn.click(
        fn=grade_mcqs_from_image,
        inputs=[image_input],
        outputs=[output]
    )

# Launch the app with proper Colab handling
try:
    from google.colab import output
    # Use standard Gradio Colab launch
    demo.launch(debug=True, share=True)
except Exception as e:
    print(f"Running outside Colab: {str(e)}")
    demo.launch()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8e06a082800aa5c9b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8e06a082800aa5c9b8.gradio.live
